# Segmenting and Clustering Neighborhoods in Toronto

### by: Jessica Jiang

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. We will scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the Toronto dataset.

## Part 1 - Data Cleaning and Setup

### **Step 1:** 
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. 
- Ignore cells with a borough that is Not assigned. 

In [142]:
import pandas as pd
source = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = source[0]
df = df[~(df.Borough=='Not assigned')]
df.reset_index().drop(['index'],axis =1)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Etobicoke,Islington Avenue
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


### **Step 2:** 
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [63]:
df_group = pd.DataFrame(df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index())

In [143]:
df = df_group.rename(columns = {"Postcode": "PostalCode","Neighbourhood":"Neighborhood"})

In [144]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [145]:
df.shape

(103, 3)

## Part 2 - adding latitude and longtitude

In [55]:
!conda install -c conda-forge geocoder --yes
print("Installation Done!")
import geocoder # import geocoder
print("Geo Coder imported!")

Solving environment: done

# All requested packages already installed.

Installation Done!
Geo Coder imported!


In [146]:
def get_geocoder(postal_code_from_df):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df.strip()))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude,longitude

In [147]:
df['Latitude'], df['Longitude'] = zip(*df['PostalCode'].apply(get_geocoder))
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


## Part 3 - Explore and cluster the neighborhoods in Toronto
You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

**Just make sure:**
- to add enough Markdown cells to explain what you decided to do and to report any observations you make.
- to generate maps to visualize your neighborhoods and how they cluster together.

In [78]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.21.0-py_0 conda-forge


geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [97]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge

The following packages will be DOWNGRADED:

    folium: 0.7.0-py_0 conda-forge --> 0.5.0-py_0 conda-forge


folium-0.5.0         | 45 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [148]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_ontario")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto, Ontario are 43.653963, -79.387207.


In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, label in zip(df['Latitude'], df['Longitude'], df['Borough']):
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [101]:
df.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

In [102]:
CLIENT_ID = '3JDU52VZBVVBWEF3C2Z1GNCA1K4OW1LUGBJCI3PV3JAYZFVV' # your Foursquare ID
CLIENT_SECRET = 'LNWAQOGNJFMKZY3DONHTR2FB5SBGBLLLDUETQN2P5WFI2U2P' # your Foursquare Secret
VERSION = '20200304' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3JDU52VZBVVBWEF3C2Z1GNCA1K4OW1LUGBJCI3PV3JAYZFVV
CLIENT_SECRET:LNWAQOGNJFMKZY3DONHTR2FB5SBGBLLLDUETQN2P5WFI2U2P


### Step 1: Let's explore the 1st neighborhood in our dataframe

In [150]:
df.loc[0, 'Neighborhood']

'Rouge,Malvern'

In [151]:
neigh_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neigh_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neigh_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_latitude, 
                                                               neigh_longitude))

Latitude and longitude values of Rouge,Malvern are 43.811525000000074, -79.19551746399998.


In [152]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_latitude, 
    neigh_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=3JDU52VZBVVBWEF3C2Z1GNCA1K4OW1LUGBJCI3PV3JAYZFVV&client_secret=LNWAQOGNJFMKZY3DONHTR2FB5SBGBLLLDUETQN2P5WFI2U2P&v=20200304&ll=43.811525000000074,-79.19551746399998&radius=500&limit=100'

In [153]:
import requests 
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e5ed51798205d001bc53a8a'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.81602500450008,
    'lng': -79.18929314050838},
   'sw': {'lat': 43.80702499550007, 'lng': -79.20174178749157}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5c53f7e18194fc002c76cfbb',
       'name': 'Canadian Appliance Source Whitby',
       'location': {'address': '150 Taunton Rd. W',
        'lat': 43.808353360406464,
        'lng': -79.19133067131041,
        'labeledLatL

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [154]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the **json** and structure it into a **pandas dataframe**.

In [155]:
from pandas.io.json import json_normalize 
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Canadian Appliance Source Whitby,Home Service,43.808353,-79.191331


In [156]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


### Step 2.1: Let's explore the neighborhoods in Toronto 

Let's create a **function** to **repeat the same process to all the neighborhoods.** 

In [160]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called **toronto_venues**.

In [161]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

KeyError: 'groups'

Let's check the **size of the resulting dataframe**

In [162]:
print(toronto_venues.shape)
toronto_venues.head()

(2448, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.811525,-79.195517,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
1,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725,Organized By Catherine,43.782117,-79.155239,Moving Target
2,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood,Morningside,West Hill",43.765815,-79.175193,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping
4,"Guildwood,Morningside,West Hill",43.765815,-79.175193,Heron Park Community Centre,43.768867,-79.176958,Gym / Fitness Center


In [163]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 263 uniques categories.


### Step 2.2: Analyze Each Neighborhood

In [164]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the **mean** of the frequency of occurrence of each category

In [165]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.00,0.030000,0.00,0.010000,0.000000,0.020000,0.000000,...,0.00,0.0,0.010000,0.00000,0.000000,0.000000,0.010000,0.0,0.000000,0.000000
1,Agincourt,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.000000,0.00000,0.058824,0.000000,0.000000,0.0,0.000000,0.000000
4,"Alderwood,Long Branch",0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,...,0.00,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,Bayview Village,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,"Bedford Park,Lawrence Manor East",0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,Berczy Park,0.000000,0.000000,0.00,0.000000,0.00,0.016129,0.000000,0.000000,0.000000,...,0.00,0.0,0.016129,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
8,"Birch Cliff,Cliffside West",0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
9,"Bloordale Gardens,Eringate,Markland Wood,Old B...",0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [166]:
toronto_grouped.shape

(98, 263)

Let's print each neighborhood along with the **top 5 most common venues**

In [167]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2                Hotel  0.04
3           Restaurant  0.04
4  Japanese Restaurant  0.03


----Agincourt----
                venue  freq
0       Shopping Mall  0.15
1        Skating Rink  0.08
2              Bakery  0.08
3    Department Store  0.08
4  Chinese Restaurant  0.08


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                       venue  freq
0           Sushi Restaurant   0.5
1                   Pharmacy   0.5
2  Middle Eastern Restaurant   0.0
3              Moving Target   0.0
4              Movie Theater   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.12
1   Fried Chicken Joint  0.06
2  Fast Food Restaurant  0.06
3          Liquor Store  0.06
4              Gym Pool  0.06


----Alderwood,Long Branch----
        

### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in **descending order.**

In [168]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the **top 10 venues for each neighborhood.**

In [192]:
import numpy as np

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Gastropub,Steakhouse,Japanese Restaurant,Bar
1,Agincourt,Shopping Mall,Park,Department Store,Bubble Tea Shop,Chinese Restaurant,Sushi Restaurant,Bakery,Discount Store,Supermarket,Pool
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Sushi Restaurant,Pharmacy,Women's Store,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Park,Sushi Restaurant,Japanese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Caribbean Restaurant,Coffee Shop,Sandwich Place,Beer Store
4,"Alderwood,Long Branch",Convenience Store,Gym,Pub,Athletics & Sports,Field,Fast Food Restaurant,Farmers Market,Farm,Fish & Chips Shop,Donut Shop


### Step 3: Cluster Neighborhoods

Run k-means to cluster the neighborhood into **5** clusters.

In [193]:
from sklearn.cluster import KMeans 

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 1, 2, 2, 1, 2], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [194]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted=neighborhoods_venues_sorted.set_index('Neighborhood')

df_right = pd.merge(df, neighborhoods_venues_sorted, on='Neighborhood', how='right')
df_right.head(2)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517,2,Home Service,Women's Store,Farm,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725,2,Bar,Moving Target,Women's Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [196]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_right['Latitude'], df_right['Longitude'], df_right['Neighborhood'], df_right['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters